In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(3)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/4.vehicle'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

from tqdm import tqdm

# = = = = = = = = = = = = = = =

directory_in = './file/3.oem'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request, url_check = f'''https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:{crawler_series['Part Number']})&size=5000&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw''', crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if not resp.json():
                raise

            # = = = = = = = = = = = = = = =

            list_make, list_model, list_submodel, list_year, list_list_engine = [dict_['fitment']['Vcdb']['MakeName'].strip() for dict_ in resp.json()], [dict_['fitment']['Vcdb']['ModelName'].strip() for dict_ in resp.json()], [dict_['fitment']['Vcdb']['SubModelName'].strip() for dict_ in resp.json()], [int(dict_['fitment']['Vcdb']['Year'].strip()) for dict_ in resp.json()], [dict_['fitment']['Vcdb']['Engines'] for dict_ in resp.json()]

            # = = = = = = = = = = = = = = =

            list_dict_correct_temp = []
            for i, list_engine in enumerate(list_list_engine):
                for engine in list_engine:
                    liter = f'{engine['Liter']}' if 'Liter' in engine else ''
                    liter = '' if liter == '' else f'{liter}L' if '.' in liter else f'{liter}.0L'

                    cylinder = f'{engine['BlockType'].strip()}{engine['Cylinders']}' if 'BlockType' in engine and 'Cylinders' in engine else ''

                    cid, cc = f'{engine['CID']}cid' if 'CID' in engine else '-cid', f'{engine['CC']}cc' if 'CC' in engine else '-cc'
                    if cid == '-cid' and cc != '-cc':
                        displacement = cc
                    elif cid != '-cid' and cc == '-cc':
                        displacement = cid
                    else:
                        displacement = f'{cid}/{cc}'

                    aspiration = engine['AspirationName'].strip() if 'AspirationName' in engine else ''

                    new_engine = re.sub(r' +', ' ', f'{liter} {cylinder} {displacement} {aspiration}')
                    list_dict_correct_temp.append({'Part Number': crawler_series['Part Number'],
                                                   'Make': list_make[i],
                                                   'Model': list_model[i],
                                                   'Submodel': list_submodel[i],
                                                   'Engine': new_engine,
                                                   'Year': list_year[i]})

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Number', 'Make', 'Model', 'Submodel', 'Engine', 'Year'],
                                                                 ascending=[True, True, True, True, True, False],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[False] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/25 17:05:44] - 2. https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-10000)&size=5000&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/25 17:05:44] - 6. https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-10009)&size=5000&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/25 17:05:44] - 8. https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-10013)&size=5000&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/25 17:05:44] - 5. https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-10007)&size=5000&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6V

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3656.76it/s]

Done ~
